In [1]:
from torch.utils.data import Dataset
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cross_validation import train_test_split
from pretrainedmodels.models import bninception
from imgaug import augmenters as iaa

import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import time

import matplotlib.pyplot as plt
from PIL import Image
import pickle

import matplotlib.pyplot as plt # plt 用于显示图片
import matplotlib.image as mpimg # mpimg 用于读取图片
import numpy as np
import copy

from preprocess import *

img_w, img_h = 64, 64
random_seed = 4050
config_batch_size = 4
class_n = (9 + 10 + 26)
output_n = 9
num_epochs = 100
feature_extract = True
use_pretrained=True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

char_to_index = {"深":0, "秦":1, "京":2, "海":3, "成":4, "南":5, "杭":6, "苏":7, "松":8}
print(char_to_index)

/home/lbyg/anaconda3/envs/pytorch/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


{'深': 0, '秦': 1, '京': 2, '海': 3, '成': 4, '南': 5, '杭': 6, '苏': 7, '松': 8}


In [2]:
class CarIdDataset(Dataset):
    def __init__(self, data_list, mode, weight = 229, height = 229):
        self.data_list = data_list
        self.mode =mode
        self.weight = weight
        self.height = height

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self,index):
        img_path, label = self.data_list[index]["image_path"], self.data_list[index]["label"]
        img = np.array(Image.open(img_path))
        
        h, w, _ = img.shape
        M = cv2.getAffineTransform(self.data_list[index]["pts"][0], self.data_list[index]["pts"][1])
        img_dst = cv2.warpAffine(img, M, (w, h))
        
        #print("================================")
        char_img_list = []
        for [x, y] in self.data_list[index]["char_segmentation"]:
            char_img = cv2.resize(img_dst[:, x:y, :], (img_w, img_h), interpolation=cv2.INTER_CUBIC)
            augment_img = iaa.SomeOf(2, [
                iaa.Affine(rotate=(-30, 30), shear=(-16, 16), translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}),
                iaa.GaussianBlur(sigma=(0.0, 3.0)),
                iaa.AdditiveGaussianNoise(scale=0.5*255),
                iaa.Add((-40, 40), per_channel=0.5),
                iaa.Sharpen(alpha=0.5),
                iaa.CropAndPad(percent=(-0.25, 0.25)),
            ])
            if self.mode == "train":
                char_img = augment_img.augment_image(char_img)
            char_img = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])(char_img)
            #print("type(char_img) = ", type(char_img))
            #print("char_img.shape = ", char_img.shape)
            char_img_list.append(char_img)
        img = torch.stack(char_img_list,0)
        #img = torch.cat(inputs=char_img_list, dimension=0)
        #print("img.shape = ", img.shape)
        
        #img = cv2.resize(img,(self.weight, self.height))
        img = transforms.Compose([])(img)
        #print(img.shape)
        y = np.zeros((output_n, class_n))
        for i in range(len(label)):
            y[i, label[i]] = 1
        
        return img, y

In [3]:
label_file = "./data/train-data-label.txt"
image_file = "./data/train-data"
model_file = "./model"
data_list = []
with open(label_file, 'r') as file_to_read:
    while True:
        lines = file_to_read.readline().strip() # 整行读取数据
        if not lines:
            break
        lines = lines.split(",  ")
        image_path = os.path.join(image_file, lines[1])
        label = [];
        label.append(char_to_index[lines[0][0]])
        for i in range(1, len(lines[0])):
            if '0' <= lines[0][i] and lines[0][i] <= '9':
                label.append(9 + ord(lines[0][i]) - ord('0'))
            else:
                label.append(9 + 10 + ord(lines[0][i]) - ord('A'))
        data_list.append({"image_path": image_path, "label":label})

In [4]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                inputs = torch.cat([inputs[i] for i in range(inputs.shape[0])], 0)
                labels = torch.cat([labels[i] for i in range(labels.shape[0])], 0)
                #print("type(inputs) = ", type(inputs))
                #print("type(labels) = ", type(labels))
                #print("inputs.shape = ", inputs.shape)
                #print("labels.shape = ", labels.shape)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs).double()
                    outputs = outputs.squeeze()
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)
                    _, y = torch.max(labels, 1)
                    #print("preds = ", preds)
                    #print("labels = ", labels)
                    running_corrects += torch.sum(preds == y)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                #running_corrects += torch.sum(preds == labels.data)

            #print(len(dataloaders[phase].dataset))
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                model_filename = model_file + os.sep + str(epoch) + "checkpoint.pth.tar"
                torch.save({"state_dict":best_model_wts}, model_filename)
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            model_filename = model_file + os.sep + "last_checkpoint.pth.tar"
            torch.save({"state_dict":model.state_dict()}, model_filename)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    best_model_filename = model_file + os.sep + "best_checkpoint.pth.tar"
    torch.save({"state_dict":best_model_wts}, best_model_filename)
    final_model_filename = model_file + os.sep + "final_checkpoint.pth.tar"
    torch.save({"state_dict":model.state_dict()}, final_model_filename)
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [5]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [7]:
#data_list = data_list[:100]
"""
for count in range(len(data_list)):
    data = data_list[count]
    print("count = ", count, "data = ", data)
    pts1, pts2, char_segmentation = preprocess(data["image_path"])
    data["pts"] = [pts1, pts2]
    data["char_segmentation"] = char_segmentation
    
    data_pkl_path = data["image_path"].replace("jpg", "pkl").replace("train-data", "preprocess")
    output = open(data_pkl_path, 'wb')
    pickle.dump(data, output)
    output.close()
"""

'\nfor count in range(len(data_list)):\n    data = data_list[count]\n    print("count = ", count, "data = ", data)\n    pts1, pts2, char_segmentation = preprocess(data["image_path"])\n    data["pts"] = [pts1, pts2]\n    data["char_segmentation"] = char_segmentation\n    \n    data_pkl_path = data["image_path"].replace("jpg", "pkl").replace("train-data", "preprocess")\n    output = open(data_pkl_path, \'wb\')\n    pickle.dump(data, output)\n    output.close()\n'

In [13]:
"""
import pickle

preprocess_data_list = []
for data in data_list:
    data_pkl_path = data["image_path"].replace("jpg", "pkl").replace("train-data", "preprocess")
    pkl_file = open(data_pkl_path, 'rb')
    pkl_data = pickle.load(pkl_file)
    preprocess_data_list.append(pkl_data)

output = open('./data.pkl', 'wb')
pickle.dump(preprocess_data_list, output)
output.close()
"""

In [ ]:
"""
for data in data_list:
    flag = 0
    for [x, y] in data["char_segmentation"]:
        if x == y:
            flag = 1
    if flag == 1:
        print(data["char_segmentation"])
        pts1, pts2, char_segmentation = preprocess(data["image_path"])
        data["pts"] = [pts1, pts2]
        data["char_segmentation"] = char_segmentation
        print(data["char_segmentation"])
output = open('./data.pkl', 'wb')
pickle.dump(data_list, output)
output.close()
"""

In [6]:
import pickle

pkl_file = open('./data.pkl', 'rb')

data_list = pickle.load(pkl_file)
print(len(data_list))

4000


In [7]:
config_batch_size = 32
train_data_list, val_data_list, _, _ = train_test_split(data_list, data_list, test_size=0.2, random_state=random_seed)
train_gen = CarIdDataset(train_data_list, "train")
train_loader = DataLoader(train_gen,batch_size=config_batch_size,shuffle=True,pin_memory=True,num_workers=2)

val_gen = CarIdDataset(val_data_list, "val")
val_loader = DataLoader(val_gen,batch_size=config_batch_size,shuffle=False,pin_memory=True,num_workers=2)
dataloaders_dict = {"train":train_loader, "val":val_loader}

In [8]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.feature = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(2, 2)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
            torch.nn.Conv2d(64, 192, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
            torch.nn.Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            torch.nn.ReLU(),
            torch.nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            torch.nn.ReLU(),
            torch.nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        self.classify = nn.Sequential(
                nn.BatchNorm1d(256),
                nn.Dropout(0.5),
                nn.Linear(256, 45),
            )

    def forward(self, x):
        x = self.feature(x)
        adaptiveAvgPoolWidth = x.shape[2]
        x = F.avg_pool2d(x, kernel_size=adaptiveAvgPoolWidth)
        x = x.view(x.size(0), -1)
        x = self.classify(x)
        return x

In [10]:
"""
model_ft = models.alexnet(pretrained=use_pretrained)
set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs,class_n)
model_ft.to(device)
print(model_ft)
model_ft = torch.nn.Sequential(
    torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(2, 2)),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
    torch.nn.Conv2d(64, 192, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1)),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
    torch.nn.Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(),
    torch.nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(),
    torch.nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
    torch.nn.Dropout(p=0.5),
    torch.nn.Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0)),
    torch.nn.ReLU(),
    torch.nn.Conv2d(128, class_n, kernel_size=(1, 1), stride=(1, 1), padding=(0, 0)),
)
"""
model_ft = Net()
last_model_filename = model_file + os.sep + "last_checkpoint.pth.tar"
last_model = torch.load(last_model_filename)
model_ft.load_state_dict(last_model["state_dict"])
model_ft.to(device)

params_to_update = model_ft.parameters()
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.BCEWithLogitsLoss().to(device)

model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=200)

Epoch 0/199
----------
train Loss: 0.6021 Acc: 5.0056
val Loss: 0.4234 Acc: 7.3413

Epoch 1/199
----------
train Loss: 0.6025 Acc: 5.0084
val Loss: 0.4270 Acc: 7.3575

Epoch 2/199
----------
train Loss: 0.6001 Acc: 5.0497
val Loss: 0.4244 Acc: 7.3500

Epoch 3/199
----------
train Loss: 0.5993 Acc: 5.0350
val Loss: 0.4214 Acc: 7.3662

Epoch 4/199
----------
train Loss: 0.5986 Acc: 5.0312
val Loss: 0.4167 Acc: 7.3563

Epoch 5/199
----------
train Loss: 0.5987 Acc: 5.0356
val Loss: 0.4184 Acc: 7.3813

Epoch 6/199
----------
train Loss: 0.5964 Acc: 5.0294
val Loss: 0.4172 Acc: 7.3700

Epoch 7/199
----------
train Loss: 0.5953 Acc: 5.0600
val Loss: 0.4183 Acc: 7.3875

Epoch 8/199
----------
train Loss: 0.5951 Acc: 5.0906
val Loss: 0.4166 Acc: 7.3925

Epoch 9/199
----------
train Loss: 0.5932 Acc: 5.0784
val Loss: 0.4122 Acc: 7.3700

Epoch 10/199
----------
train Loss: 0.5913 Acc: 5.0812
val Loss: 0.4091 Acc: 7.3975

Epoch 11/199
----------
train Loss: 0.5912 Acc: 5.1053
val Loss: 0.4123 Acc

val Loss: 0.3267 Acc: 7.7800

Epoch 97/199
----------
train Loss: 0.5146 Acc: 5.7241
val Loss: 0.3292 Acc: 7.8013

Epoch 98/199
----------
train Loss: 0.5141 Acc: 5.7194
val Loss: 0.3232 Acc: 7.7825

Epoch 99/199
----------
train Loss: 0.5146 Acc: 5.7266
val Loss: 0.3267 Acc: 7.7875

Epoch 100/199
----------
train Loss: 0.5128 Acc: 5.7344
val Loss: 0.3258 Acc: 7.7888

Epoch 101/199
----------
train Loss: 0.5130 Acc: 5.7053
val Loss: 0.3246 Acc: 7.7862

Epoch 102/199
----------
train Loss: 0.5126 Acc: 5.7281
val Loss: 0.3243 Acc: 7.7938

Epoch 103/199
----------
train Loss: 0.5109 Acc: 5.7694
val Loss: 0.3247 Acc: 7.7925

Epoch 104/199
----------
train Loss: 0.5086 Acc: 5.7525
val Loss: 0.3220 Acc: 7.7963

Epoch 105/199
----------
train Loss: 0.5115 Acc: 5.7438
val Loss: 0.3218 Acc: 7.7975

Epoch 106/199
----------
train Loss: 0.5104 Acc: 5.7366
val Loss: 0.3187 Acc: 7.8025

Epoch 107/199
----------
train Loss: 0.5078 Acc: 5.7737
val Loss: 0.3208 Acc: 7.8150

Epoch 108/199
----------
tr

train Loss: 0.4547 Acc: 6.1794
val Loss: 0.2659 Acc: 7.9725

Epoch 193/199
----------
train Loss: 0.4541 Acc: 6.1681
val Loss: 0.2661 Acc: 7.9787

Epoch 194/199
----------
train Loss: 0.4544 Acc: 6.1634
val Loss: 0.2648 Acc: 7.9850

Epoch 195/199
----------
train Loss: 0.4543 Acc: 6.1541
val Loss: 0.2649 Acc: 7.9725

Epoch 196/199
----------
train Loss: 0.4536 Acc: 6.1584
val Loss: 0.2633 Acc: 7.9800

Epoch 197/199
----------
train Loss: 0.4513 Acc: 6.1791
val Loss: 0.2640 Acc: 7.9850

Epoch 198/199
----------
train Loss: 0.4531 Acc: 6.1531
val Loss: 0.2637 Acc: 7.9787

Epoch 199/199
----------
train Loss: 0.4509 Acc: 6.1919
val Loss: 0.2637 Acc: 7.9725

Training complete in 62m 25s
Best val Acc: 7.985000


In [10]:
a = torch.zeros(2, 3)
print(a)
b = torch.zeros(2, 3)
print(b)
c = torch.stack((a,b),0)
print(c.shape)
print(c[0])

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
torch.Size([2, 2, 3])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
